In [1]:
import copy
import tensorflow as tf
from typing import Union

from c3.c3objs import C3obj, Quantity
from c3.experiment import Experiment
from c3.parametermap import ParameterMap
from c3.libraries import algorithms, constants, envelopes, fidelities
from c3.generator import devices
from c3.signal import pulse, gates
from c3.generator.generator import Generator
from c3.optimizers.optimalcontrol import OptimalControl

In [2]:
D_gs = 2.87 * 1e3  # in GHz

gamma_NV = 28.0 * 1e3
omega_0 = np.sqrt(2) * gamma_NV

Gauss_to_Tesla = 0.0001

B_0 = 6000.0  # in Gauss
B_0 = B_0 * Gauss_to_Tesla

delta_0 = D_gs - gamma_NV * B_0
""" TODO: The values of the NV center still need to be converted to
        physically correct units!"
"""
freq_NV_val = delta_0  # -14490.0
freq_N15_val = 3.0e6  # .* 2*pi # in MHz (Hyperfine Splittings)
freq_C13_val = 0.413e6  # .* 2*pi # in MHz (Hyperfine Splittings)

# just some offsets randomly chosen for the computation of
# the minimum and maximum physical limits of the quantity
# see below
sigma_freq_NV = 500
sigma_freq_N15 = 0.05e6
sigma_freq_C13 = 0.5e6

# gyromagnetic ratios
gyromag_ratio_NV_val = 28.0 * 1e3  # GHz/T
gyromag_ratio_N15_val = 3.077e6  # gamma_n/(2 * pi) (MHz * T^-1)
gyromag_ratio_C13_val = 10.7084e6  # gamma_n/(2 * pi) (MHz * T^-1)

# just some offsets randomly chosen for the computation of
# the minimum and maximum physical limits of the quantity
# see below
sigma_gyromag_ratio_NV = 5e3
sigma_gyromag_ratio_N15 = 0.5e6
sigma_gyromag_ratio_C13 = 2.0e6

In [3]:
class NVModel(C3obj):
    """
    Represents the element in a chip functioning as qubit.

    Parameters
    ----------
    freq: Quantity
        frequency of the qubit
    anhar: Quantity
        anharmonicity of the qubit. defined as w01 - w12
    t1:Quantity
        t1, the time decay of the qubit due to dissipation
    t2star: Quantity
        t2star, the time decay of the qubit due to pure dephasing
    temp: Quantity
        temperature of the qubit, used to determine the Boltzmann distribution
        of energy level populations

    """

    def __init__(
        self,
        name,
        hilbert_dim=8,  # set const to 8 here
        desc=None,
        comment=None,
        freq_NV: Quantity = None,
        freq_N15: Quantity = None,
        freq_C13: Quantity = None,
        gyromag_ratio_NV: Quantity = None,
        gyromag_ratio_N15: Quantity = None,
        gyromag_ratio_C13: Quantity = None,
        params=None,
    ):
        super().__init__(
            name=name, desc=desc, comment=comment, params=params,
        )
        self.hilbert_dim = hilbert_dim
        self.couplings = []
        self.subsystems = []
        self.tasks = []
        self.lindbladian = False
        self.use_FR = True
        self.controllability = False
        self.max_excitations = 0
        self.dephasing_strength = 0.0
        self.state_labels = [
            "|000>",
            "|001>",
            "|010>",
            "|011>",
            "|100>",
            "|101>",
            "|110>",
            "|111>",
        ]

        self.names = ["NV", "N15", "C13"]
        self.dims = [2, 2, 2]

        if freq_NV:
            self.params["freq_NV"] = freq_NV
        if freq_N15:
            self.params["freq_N15"] = freq_N15
        if freq_C13:
            self.params["freq_C13"] = freq_C13
        if gyromag_ratio_NV:
            self.params["gyromag_ratio_NV"] = gyromag_ratio_NV
        if gyromag_ratio_N15:
            self.params["gyromag_ratio_N15"] = gyromag_ratio_N15
        if gyromag_ratio_C13:
            self.params["gyromag_ratio_C13"] = gyromag_ratio_C13

        self.init_Hs()

    def init_Hs(self):
        """
        Initialize the qubit Hamiltonians. If the dimension is higher than two, a
        Duffing oscillator is used.

        Parameters
        ----------
        ann_oper : np.array
            Annihilation operator in the full Hilbert space

        """

        # Pauli matrices definitions as they are imported from the c3 libraries
        # in the import section above (here as comment just for illustration):
        # Id = np.array([[1, 0], [0, 1]], dtype=np.complex128)
        # X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
        # Y = np.array([[0, -1j], [1j, 0]], dtype=np.complex128)
        # Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
        #
        # PAULIS = {"X": X, "Y": Y, "Z": Z, "Id": Id}

        PAULIS = constants.PAULIS

        Id = PAULIS["Id"]
        X = PAULIS["X"]
        Y = PAULIS["Y"]
        Z = PAULIS["Z"]

        self.Ix1 = tf.constant(np.kron(X, np.kron(Id, Id)), dtype=tf.complex128)
        self.Ix2 = tf.constant(np.kron(Id, np.kron(X, Id)), dtype=tf.complex128)
        self.Ix3 = tf.constant(np.kron(Id, np.kron(Id, X)), dtype=tf.complex128)

        self.Iy1 = tf.constant(np.kron(Y, np.kron(Id, Id)), dtype=tf.complex128)
        self.Iy2 = tf.constant(np.kron(Id, np.kron(Y, Id)), dtype=tf.complex128)
        self.Iy3 = tf.constant(np.kron(Id, np.kron(Id, Y)), dtype=tf.complex128)

        self.Iz1 = tf.constant(np.kron(Z, np.kron(Id, Id)), dtype=tf.complex128)
        self.Iz2 = tf.constant(np.kron(Id, np.kron(Z, Id)), dtype=tf.complex128)
        self.Iz3 = tf.constant(np.kron(Id, np.kron(Id, Z)), dtype=tf.complex128)


    def get_Hamiltonian(
        self, signal: Union[dict, bool] = None, transform: tf.Tensor = None
    ):

        t = tf.cast(signal["d1"]["ts"], tf.complex128)

        sig = tf.zeros_like(t)
        for channel in signal.keys():
            sig += tf.cast(signal[channel]["values"], tf.complex128)

        # freq_NV = self.params["freq_NV"].get_value(dtype=tf.complex128) # Unused for now
        zero = tf.constant(0.0, dtype=tf.float64)
        freq_N15 = tf.complex(self.params["freq_N15"].get_value(), zero)
        freq_C13 = tf.complex(self.params["freq_C13"].get_value(), zero)
        gyro_N15 = tf.complex(self.params["gyromag_ratio_N15"].get_value(), zero)
        gyro_C13 = tf.complex(self.params["gyromag_ratio_C13"].get_value(), zero)

        p1 = tf.expand_dims(self.Ix2, axis=0) * tf.expand_dims(
            tf.expand_dims(tf.cos(freq_N15 * t), axis=1), axis=1
        )
        p2 = tf.expand_dims(self.Iy2, axis=0) * tf.expand_dims(
            tf.expand_dims(tf.sin(freq_N15 * t), axis=1), axis=1
        )
        h_N15 = p1 + p2

        p3 = tf.expand_dims(self.Ix3, axis=0) * tf.expand_dims(
            tf.expand_dims(tf.cos(freq_C13 * t), axis=1), axis=1
        )
        p4 = tf.expand_dims(self.Iy3, axis=0) * tf.expand_dims(
            tf.expand_dims(tf.sin(freq_C13 * t), axis=1), axis=1
        )
        h_C13 = p3 + p4

        sig = tf.expand_dims(tf.expand_dims(sig, axis=1), axis=1)
        h =-(h_N15 * gyro_N15 + h_C13 * gyro_C13)* sig

        return h

    def get_Frame_Rotation(self, t_final: np.float64, freqs: dict, framechanges: dict):
        """
        Compute the frame rotation needed to align Lab frame and rotating Eigenframes
        of the qubits.

        Parameters
        ----------
        t_final : tf.float64
            Gate length
        freqs : list
            Frequencies of the local oscillators.
        framechanges : list
            List of framechanges. A phase shift applied to the control signal to
            compensate relative phases of drive oscillator and qubit.

        Returns
        -------
        tf.Tensor
            A (diagonal) propagator that adjust phases
        """
        Z = constants.PAULIS["Z"]
        num_oper = tf.constant(np.kron(Z, np.kron(Z, Z)), dtype=tf.complex128)
        exponent = 0
        for channel in freqs.keys():
            freq = freqs[channel]
            framechange = framechanges[channel]
            exponent += 1.0j * num_oper * (freq * t_final + framechange)
        if len(exponent.shape) == 0:
            return tf.eye(self.tot_dim, dtype=tf.complex128)
        FR = tf.linalg.expm(exponent)
        return FR

In [4]:
model = NVModel(
    name="N15-C13 Qubit",
    desc="Qubit 1",
    comment="The one and only qubit in this chip",
    freq_NV=Quantity(
        value=freq_NV_val,
        min_val=freq_NV_val - sigma_freq_NV,
        max_val=freq_NV_val + sigma_freq_NV,
        unit="Hz 2pi",  # WARNING: The unit is important at this location
                        # as it will search for any occurance of '2pi'
                        # and automatically internally multiply the
                        # factor on top. This guarantees human readable
                        # units.
    ),
    freq_N15=Quantity(
        value=freq_N15_val,
        min_val=freq_N15_val - sigma_freq_N15,
        max_val=freq_N15_val + sigma_freq_N15,
        unit="Hz 2pi",
    ),
    freq_C13=Quantity(
        value=freq_C13_val,
        min_val=freq_C13_val - sigma_freq_C13,
        max_val=freq_C13_val + sigma_freq_C13,
        unit="Hz 2pi",
    ),
    gyromag_ratio_NV=Quantity(
        value=gyromag_ratio_NV_val,
        min_val=gyromag_ratio_NV_val - sigma_gyromag_ratio_NV,
        max_val=gyromag_ratio_NV_val + sigma_gyromag_ratio_NV,
        unit="Hz 2pi/T",
    ),
    gyromag_ratio_N15=Quantity(
        value=gyromag_ratio_N15_val,
        min_val=gyromag_ratio_N15_val - sigma_gyromag_ratio_N15,
        max_val=gyromag_ratio_N15_val + sigma_gyromag_ratio_N15,
        unit="Hz 2pi/T",
    ),
    gyromag_ratio_C13=Quantity(
        value=gyromag_ratio_C13_val,
        min_val=gyromag_ratio_C13_val - sigma_gyromag_ratio_C13,
        max_val=gyromag_ratio_C13_val + sigma_gyromag_ratio_C13,
        unit="Hz 2pi/T",
    ),
)

2022-08-11 15:20:29.853225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.controllability

False

In [6]:
sim_res = 10e9  # Resolution for numerical simulation
awg_res = 10e9  # Realistic, limited resolution of an AWG
lo = devices.LO(name="lo", resolution=sim_res)
awg = devices.AWG(name="awg", resolution=awg_res)
mixer = devices.Mixer(name="mixer")

# Waveform generators exhibit a rise time, the time
# it takes until the target voltage is set. This has
# a smoothing effect on the resulting pulse shape.
resp = devices.ResponseFFT(
    name="resp",
    rise_time=Quantity(value=0.3e-9, min_val=0.05e-9, max_val=0.6e-9, unit="s"),
    resolution=sim_res,
)

# In simulation, we translate between AWG resolution
# and simulation (or "analog") resolultion by including
# an up-sampling device.
dig_to_an = devices.DigitalToAnalog(name="dac", resolution=sim_res)

# Control electronics apply voltages to lines, whereas
# in a Hamiltonian we usually write the control fields
# in energy or frequency units. In practice, this
# conversion can be highly non-trivial if it involves
# multiple stages of attenuation and for example the
# conversion of a line voltage in an antenna to a
# dipole field coupling to the qubit. The following
# device represents a simple, linear conversion factor.
v2Hz = np.pi / (2.0 * (gyromag_ratio_C13_val * 2 * np.pi)) * 1e6  # ~1/12

v_to_Hz = devices.VoltsToHertz(
    name="v_to_Hz", V_to_Hz=Quantity(value=v2Hz, min_val=1e-6, max_val=1.0, unit="Hz/V")
)
device_dict = {
    "LO": lo,
    "AWG": awg,
    "Mixer": mixer,
    "VoltsToHertz": v_to_Hz,
    "DigitalToAnalog": dig_to_an,
    "Response": resp,
}

chain_d1 = {
    "d1":{
        "LO": [],
        "AWG": [],
        "DigitalToAnalog": ["AWG"],
        "Mixer": ["LO", "DigitalToAnalog"],
        "VoltsToHertz": ["Mixer"]
    },
    "d2": {
        "LO": [],
        "AWG": [],
        "DigitalToAnalog": ["AWG"],
        "Mixer": ["LO", "DigitalToAnalog"],
        "VoltsToHertz": ["Mixer"]
    }
}
generator = Generator(devices=device_dict, chains=chain_d1)

In [7]:
amp_val = 1.0

t_final = 1e-6  # Time for single qubit gates
gauss_params_single = {
    "amp": Quantity(value=amp_val, min_val=-0.01, max_val=5.0, unit="V"),  # 0.5,
    "t_final": Quantity(
        value=t_final, min_val=0.5 * t_final, max_val=1.5 * t_final, unit="s"
    ),
    "sigma": Quantity(
        value=t_final / 4, min_val=t_final / 8, max_val=t_final / 2, unit="s"
    ),
    "xy_angle": Quantity(
        value=0.0, min_val=-0.5 * np.pi, max_val=2.5 * np.pi, unit="rad"
    ),
    "freq_offset": Quantity(
        value=0.001, min_val=-5.0 * 1e6, max_val=+5.0 * 1e6, unit="Hz 2pi"
    ),
}

gauss_env_single = pulse.Envelope(
    name="gauss",
    desc="Gaussian comp for single-qubit gates",
    params=gauss_params_single,
    shape=envelopes.gaussian_nonorm
)

nodrive_env = pulse.Envelope(
    name="no_drive",
    params={
        "t_final": Quantity(
            value=t_final, min_val=0.5 * t_final, max_val=1.5 * t_final, unit="s"
        )
    },
    shape=envelopes.no_drive,
)

carrier_parameters = {
    "freq": Quantity(value=freq_C13_val, min_val=0.0, max_val=1e9, unit="Hz 2pi"),
    "framechange": Quantity(value=0.001, min_val=-np.pi, max_val=3 * np.pi, unit="rad"),
}
carr_C13 = pulse.Carrier(
    name="carrier",
    desc="Frequency of the local oscillator",
    params=carrier_parameters,
)
carr_N15 = copy.deepcopy(carr_C13)
carr_N15.params["freq"].set_value(freq_N15_val)

rx90p_C13 = gates.Instruction(
    name="rx90p", targets=[2], t_start=0.0, t_end=t_final, channels=["d1", "d2"]
)

# Using drive d1 for C13 and d2 for N15
# We define all instructions with both carriers to enable simultaneous driving
rx90p_C13.add_component(gauss_env_single, "d1")
rx90p_C13.add_component(carr_C13, "d1")
rx90p_C13.add_component(copy.deepcopy(gauss_env_single), "d2")
rx90p_C13.add_component(carr_N15, "d2")

ry90p_C13 = copy.deepcopy(rx90p_C13)
ry90p_C13.name = "ry90p"
rx90m_C13 = copy.deepcopy(rx90p_C13)
rx90m_C13.name = "rx90m"
ry90m_C13 = copy.deepcopy(rx90p_C13)
ry90m_C13.name = "ry90m"
ry90p_C13.comps["d1"]["gauss"].params["xy_angle"].set_value(0.5 * np.pi)
rx90m_C13.comps["d1"]["gauss"].params["xy_angle"].set_value(np.pi)
ry90m_C13.comps["d1"]["gauss"].params["xy_angle"].set_value(1.5 * np.pi)

rx90p_N15 = copy.deepcopy(rx90p_C13)
rx90p_N15.targets = [1]

# Setting the respective other resonant drive to 0
rx90p_C13.comps["d2"]["gauss"].params["amp"].set_value(0)

rx90p_N15.comps["d1"]["gauss"].params["amp"].set_value(0)
rx90p_N15.comps["d2"]["gauss"].params["amp"].set_value(
    5.0
)  # Due to gyromag, N15 needs higher amp

ry90p_N15 = copy.deepcopy(rx90p_N15)
ry90p_N15.name = "ry90p"
rx90m_N15 = copy.deepcopy(rx90p_N15)
rx90m_N15.name = "rx90m"
ry90m_N15 = copy.deepcopy(rx90p_N15)
ry90m_N15.name = "ry90m"
ry90p_N15.comps["d2"]["gauss"].params["xy_angle"].set_value(0.5 * np.pi)
rx90m_N15.comps["d2"]["gauss"].params["xy_angle"].set_value(np.pi)
ry90m_N15.comps["d2"]["gauss"].params["xy_angle"].set_value(1.5 * np.pi)

single_q_gates = [
    rx90p_C13,
    ry90p_C13,
    rx90m_C13,
    ry90m_C13,
    rx90p_N15,
    ry90p_N15,
    rx90m_N15,
    ry90m_N15,
]

In [8]:
parameter_map = ParameterMap(instructions=single_q_gates, model=model, generator=generator)
exp = Experiment(pmap=parameter_map, sim_res=sim_res)
exp.use_control_fields = False

In [10]:
exp.set_opt_gates(["rx90p[2]"])
unitaries = exp.compute_propagators()

In [9]:
#opt_gates = ["rx90p[1]"]
opt_gates = ["rx90p[2]"]
# opt_gates = ["rx90p[1]"], "rx90p[2]"]

gateset_opt_map = [
    # [("rx90p[1]", "d2", "gauss", "amp"),],
    # [("rx90p[1]", "d2", "gauss", "freq_offset"),],
    # [("rx90p[1]", "d2", "carrier", "framechange")]
    # [
    #   ("rx90p[1]", "d2", "gauss", "xy_angle"),
    # ],
    # [
    #   ("rx90p[1]", "d2", "gauss", "delta"),
    # ],
    [("rx90p[2]", "d1", "gauss", "amp"),],
    [("rx90p[2]", "d1", "gauss", "freq_offset"),],
    [("rx90p[2]", "d1", "carrier", "framechange")],
    # [
    #   ("rx90p[2]", "d1", "gauss", "xy_angle"),
    # ],
    # [
    #   ("rx90p[2]", "d1", "gauss", "delta"),
    # ],
]


parameter_map = exp.pmap
parameter_map.set_opt_map(gateset_opt_map)
parameter_map.print_parameters()


# Create a temporary directory to store logfiles, modify as needed
# log_dir = os.path.join(tempfile.TemporaryDirectory().name, "c3logs")
log_dir = os.path.join(".", "c3logs")

psi_init = [[0] * 2]
psi_init[0][0] = 1
comp_gs = tf.transpose(tf.constant(psi_init, tf.complex128))

opt = OptimalControl(
    dir_path=log_dir,
    # fid_func=fidelities.average_infid_set,
    fid_func=fidelities.state_transfer_infid_set,
    fid_func_kwargs={"psi_0": comp_gs},
    fid_subspace=["C13"],
    #fid_subspace=["N15"],
    pmap=parameter_map,
    algorithm=algorithms.lbfgs,
    options={"maxfun": 150},   # optionen für optimizer algo
    # algorithm=algorithms.cmaes,
    # options={"popsize" : 8, "maxiter" : 150},
    run_name="better_X90",
)

exp.set_opt_gates(opt_gates)
opt.set_exp(exp)

# run the optimization
opt.optimize_controls()
parameter_map.print_parameters()

rx90p[2]-d1-gauss-amp                 : 1.000 V 
rx90p[2]-d1-gauss-freq_offset         : 1.000 mHz 2pi 
rx90p[2]-d1-carrier-framechange       : 1.000 mrad 

C3:STATUS:Saving as: /home/ubuntu/c3/examples/c3logs/better_X90/2022_08_11_T_15_20_33/open_loop.c3log


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_1240358/198778774.py:59 in <cell line: 59>                                 │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1240358/198778774.py'                │
│ /home/ubuntu/c3/c3/optimizers/optimalcontrol.py:147 in optimize_controls                  │
│                                                                                           │
│   144 │   │   self.index = index                                                          │
│   145 │   │   x_init = self.pmap.get_parameters_scaled()                                  │
│   146 │   │   try:                                                                        │
│ ❱ 147 │   │   │   self.algorithm(                                                         │
│   148 │   │   │   │   x_init,                                                             │
│   149 │   │   │   │   fun=self.fct_to_min,                                                │
│   150 │   │   │   │   fun_grad=self.fct_to_min_autograd,                                  │
│                                                                                           │
│ /home/ubuntu/c3/c3/libraries/algorithms.py:418 in lbfgs                                   │
│                                                                                           │
│   415 │   """                                                                             │
│   416 │   # TODO print from the log not from here                                         │
│   417 │   # options.update({"disp": True})                                                │
│ ❱ 418 │   return minimize(                                                                │
│   419 │   │   fun_grad, x_init, jac=grad_lookup, method="L-BFGS-B", options=options       │
│   420 │   )                                                                               │
│   421                                                                                     │
│                                                                                           │
│ /home/ubuntu/.local/lib/python3.8/site-packages/scipy/optimize/_minimize.py:681 in        │
│ minimize                                                                                  │
│                                                                                           │
│   678 │   │   res = _minimize_newtoncg(fun, x0, args, jac, hess, hessp, callback,         │
│   679 │   │   │   │   │   │   │   │    **options)                                         │
│   680 │   elif meth == 'l-bfgs-b':                                                        │
│ ❱ 681 │   │   res = _minimize_lbfgsb(fun, x0, args, jac, bounds,                          │
│   682 │   │   │   │   │   │   │      callback=callback, **options)                        │
│   683 │   elif meth == 'tnc':                                                             │
│   684 │   │   res = _minimize_tnc(fun, x0, args, jac, bounds, callback=callback,          │
│                                                                                           │
│ /home/ubuntu/.local/lib/python3.8/site-packages/scipy/optimize/_lbfgsb_py.py:308 in       │
│ _minimize_lbfgsb                                                                          │
│                                                                                           │
│   305 │   │   else:                                                                       │
│   306 │   │   │   iprint = disp                                                           │
│   307 │                                                                                   │
│ ❱ 308 │   sf = _prepare_scalar_function(fun, x0, j